<a href="https://colab.research.google.com/github/shantanutrip/covid_flight_analysis/blob/master/Analysis/flight_helps_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook aims to discover how air travel helped the spread of COVID-19 

# Install dependencies

In [1]:
# Install spark-related dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

     |████████████████████████████████| 217.8MB 55kB/s 
     |████████████████████████████████| 204kB 46.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=7aca671764ee67722b4d5dc0f1f054d8e5ac1f465789f20a1aa464ecbe81a5e1
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


# Download data



In [2]:
!wget https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
!wget https://ourairports.com/data/airports.csv

--2020-05-07 07:16:16--  https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4638665 (4.4M) [text/plain]
Saving to: ‘us-counties.csv’

us-counties.csv     100%[===================>]   4.42M  14.2MB/s    in 0.3s    

2020-05-07 07:16:16 (14.2 MB/s) - ‘us-counties.csv’ saved [4638665/4638665]

--2020-05-07 07:16:17--  https://ourairports.com/data/airports.csv
Resolving ourairports.com (ourairports.com)... 206.71.179.167
Connecting to ourairports.com (ourairports.com)|206.71.179.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8475889 (8.1M) [text/csv]
Saving to: ‘airports.csv’

airports.csv        100%[===================>]   8.08M  20.3MB/s    in 0.4s    

2020-

# Execution 

In [0]:
import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

airports = spark.read.format("csv").options(header="true").load("/content/airports.csv")
disease = spark.read.format("csv").options(header="true", inferSchema="true").load("/content/us-counties.csv")
# Change merged_flight.csv path accordingly
all_flights = spark.read.format("csv").options(header="true", inferSchema="true").load("/content/drive/My Drive/Big Data Final Project/Data/merged_flight.csv")

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime

In [0]:
# Select target city(county) and state.
target_county = "Dallas"
target_state = "Texas"

In [6]:
# Finding out when COVID-19 started spreading in target city(county).
target_date = disease.filter((disease["county"] == target_county) & (disease["state"] == target_state)).orderBy("date").first()["date"]
print(target_date)

2020-03-10 00:00:00


In [7]:
# Finding out the airports' ICAO code and name for the target city(county).
target_airports = airports.filter((airports["municipality"] == target_county) & ((airports["type"] == "large_airport") | (airports["type"] == "medium_airport"))).select("ident", "name", "municipality")
target_airports.show()

+-----+-----------------+------------+
|ident|             name|municipality|
+-----+-----------------+------------+
| KDAL|Dallas Love Field|      Dallas|
+-----+-----------------+------------+



In [8]:
# Finding out all flights that landed in target city(county) in the first 4 months of 2020,
# including flights' orgin city(county).
target_flights = all_flights.filter(all_flights["origin"] != "null").join(target_airports, target_airports["ident"] == all_flights["destination"]).withColumnRenamed("ident", "dst_ident").withColumnRenamed("name", "dst_name").withColumnRenamed("municipality", "dst_municipality")
target_flights_complete = airports.select("ident", "name", "municipality").join(target_flights, target_flights["origin"] == airports["ident"]).withColumnRenamed("ident", "src_ident").withColumnRenamed("name", "src_name").withColumnRenamed("municipality", "src_municipality")
target_flights_complete.show()

+---------+--------------------+----------------+--------+------+------+------------+--------+------+-----------+--------------------+--------------------+--------------------+---------+-----------------+----------------+
|src_ident|            src_name|src_municipality|callsign|number|icao24|registration|typecode|origin|destination|           firstseen|            lastseen|                 day|dst_ident|         dst_name|dst_municipality|
+---------+--------------------+----------------+--------+------+------+------------+--------+------+-----------+--------------------+--------------------+--------------------+---------+-----------------+----------------+
|     KFLL|Fort Lauderdale H...| Fort Lauderdale| SWA3429|  null|a98520|      N712SW|    B737|  KFLL|       KDAL|2019-12-31 20:33:...|2020-01-01 00:21:...|2020-01-01 00:00:...|     KDAL|Dallas Love Field|          Dallas|
|     KLGA|  La Guardia Airport|        New York| SWA4317|  null|aa963e|      N781WN|    B737|  KLGA|       KDAL

In [9]:
# Grouping all flights by origin city(county), sort by count in descending order.
incoming_flight_info = target_flights_complete.groupBy("src_municipality", "day").count().orderBy("day", desc("count"))
incoming_flight_info.show()

+----------------+--------------------+-----+
|src_municipality|                 day|count|
+----------------+--------------------+-----+
|         Houston|2020-01-01 00:00:...|   16|
|     San Antonio|2020-01-01 00:00:...|    9|
|          Austin|2020-01-01 00:00:...|    8|
|         Chicago|2020-01-01 00:00:...|    7|
|          Denver|2020-01-01 00:00:...|    7|
|         Atlanta|2020-01-01 00:00:...|    6|
|         Phoenix|2020-01-01 00:00:...|    6|
|     Kansas City|2020-01-01 00:00:...|    6|
|        St Louis|2020-01-01 00:00:...|    6|
|         Seattle|2020-01-01 00:00:...|    5|
|       Nashville|2020-01-01 00:00:...|    5|
|     Los Angeles|2020-01-01 00:00:...|    5|
|        New York|2020-01-01 00:00:...|    5|
|      Washington|2020-01-01 00:00:...|    5|
| Fort Lauderdale|2020-01-01 00:00:...|    4|
|     Albuquerque|2020-01-01 00:00:...|    4|
|       Las Vegas|2020-01-01 00:00:...|    3|
|           Tulsa|2020-01-01 00:00:...|    3|
|        Portland|2020-01-01 00:00

In [0]:
# filtering function definition  
def date_filter(day):
  '''
  This filter checks if input date is in the two weeks range prior of the target date. 
  '''
  datetime_obj = datetime.datetime.strptime(day.split("+")[0], "%Y-%m-%d %H:%M:%S")
  two_weeks = datetime.timedelta(days=14)
  zero = datetime.timedelta(days=0)
  day_difference = target_date - datetime_obj
  if day_difference <= two_weeks and day_difference > zero:
    return True
  return False

filter_func = udf(date_filter, BooleanType())

In [11]:
# Finding out flights which landed in the target city(county) during the two weeks prior of the target date.
# The reason for only including two weeks of flight is the incubation period of COVID-19 is two weeks. 
pre_two_weeks_flights = incoming_flight_info.filter(filter_func(incoming_flight_info["day"]))
pre_two_weeks_flights.show()

+----------------+--------------------+-----+
|src_municipality|                 day|count|
+----------------+--------------------+-----+
|          Austin|2020-02-25 00:00:...|   13|
|         Atlanta|2020-02-25 00:00:...|   12|
|         Houston|2020-02-25 00:00:...|   12|
|     San Antonio|2020-02-25 00:00:...|   11|
|          Denver|2020-02-25 00:00:...|    9|
|       Las Vegas|2020-02-25 00:00:...|    9|
|     Los Angeles|2020-02-25 00:00:...|    8|
|         Chicago|2020-02-25 00:00:...|    8|
|     Kansas City|2020-02-25 00:00:...|    7|
|        St Louis|2020-02-25 00:00:...|    7|
|         Phoenix|2020-02-25 00:00:...|    6|
|      Washington|2020-02-25 00:00:...|    6|
|        New York|2020-02-25 00:00:...|    5|
|     Albuquerque|2020-02-25 00:00:...|    5|
|           Tulsa|2020-02-25 00:00:...|    4|
|         Seattle|2020-02-25 00:00:...|    4|
|       Nashville|2020-02-25 00:00:...|    4|
|       Baltimore|2020-02-25 00:00:...|    4|
|       San Diego|2020-02-25 00:00

In [12]:
# Finding out where did most of the flight came from during the two weeks time prior of the target date.
pre_two_weeks_flights.groupBy("src_municipality").sum("count").orderBy(desc("sum(count)")).show()

+----------------+----------+
|src_municipality|sum(count)|
+----------------+----------+
|         Houston|       230|
|         Atlanta|       151|
|     San Antonio|       144|
|          Austin|       140|
|       Las Vegas|       120|
|          Denver|       117|
|     Los Angeles|       109|
|         Chicago|       102|
|         Phoenix|        96|
|     Kansas City|        91|
|        St Louis|        90|
|       Baltimore|        68|
|       Nashville|        66|
|     Albuquerque|        65|
|      Washington|        65|
|        New York|        64|
|       San Diego|        57|
|         Seattle|        55|
|           Tulsa|        52|
| Fort Lauderdale|        50|
+----------------+----------+
only showing top 20 rows



# Findings

## Houston, Texas

Houston had lots of flights coming to Dallas, let's take a look at when did the spread start in Houston.

In [13]:
print(disease.filter((disease["county"] == "Houston") & (disease["state"] == "Texas")).orderBy("date").first()["date"])

2020-04-19 00:00:00


Looks like COVID-19 started spreading in Houston on April 19th, which is later than target date (March 10th), so Houston might not be the city that "infected" Dallas.

## Atlanta, Georgia

Atlanta had fair amount of flights coming to Dallas as well, let's take a look at when did the spread start in Atlanta. One problem here is that, Atlanta is the name of the city, however, in ```disease``` dataframe, county name was used instead of city name. So I had manually look up the county name of Atlanta, the county is called Fulton.

In [14]:
print(disease.filter((disease["county"] == "Fulton") & (disease["state"] == "Georgia")).orderBy("date").first()["date"])

2020-03-02 00:00:00


Hmm, looks like COVID-19 started spreading in Atlanta before it did in Dallas, therefore, it is possible that Atlanat "infected" Dallas. Additionally, given that Atlanat had the second highest amount of flights coming to Dallas, the probability is high.

## San Antonio, Texas

San Antonio had the third highest amount of flight coming to Dallas, let's take a look at when the spread started in San Antonio. The same problem happened again here, the county that San Antonio is in is called Bexar.

In [15]:
print(disease.filter((disease["county"] == "Bexar") & (disease["state"] == "Texas")).orderBy("date").first()["date"])

2020-02-12 00:00:00


Looks like COVID-19 start spreading in San Antonio way before it started in Dallas, therefore, San Antonio could have also "infected" Dallas.

# Further development

1. Achieve automatically conversion between city name and county name. (We probably should do this.)
2. Maybe we could actually calculate the probability of city *A* "infected" city *B*. We can calculate the value using the spread start date of city *A* and the number of flights from city *A*. For example, COVID-19 started spreading in Altanat on March 2nd and had 151 flights came to Dallas, on the other hand, the virus started spreading in San Antonio in February 12th and had 144 flights came to Dallas. Given these two cities, I would say that the probability of San Antonio "infected" Dallas is higher than it is for Altanta. Reason being that, even though there are slightly less flights came from San Antonio, the spread started in San Antonio really early, so by February 25th (two weeks before March 10th), there should be lots people infected in San Antonio. Therefore, chance of one of the infected individuals flew from San Antonio to Dallas during the two weeks prior of the target date is higher. (I think it's doable, but might take some time to implement, so we can do it if we have time left.)
3. Maybe some graphs? (Not really important as the result can be described in text, but might be nice to have for the presentation.)
